In [ ]:
##### REQUIRES THE DATAFRAME FOLDER TO BE NAMED 'Cohorts', WHICH INCLUDES ALL PRECOMPUTED DATAFRAMES #####
import fiber
from fiber.cohort import Cohort
from fiber.condition import Patient, MRNs
from fiber.condition import Diagnosis
from fiber.condition import Measurement, Encounter, Drug
from fiber.storage import yaml as fiberyaml
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import os
from functools import reduce 

In [ ]:
#load Cohorts
Case_filtered_15_540 = pq.read_table('Cohorts/Case/Case_filtered_15_540.parquet').to_pandas()
Control_filtered_15_540 = pq.read_table('Cohorts/Control/Control_filtered_15_540.parquet').to_pandas()

In [ ]:
#define test Cohorts
Test_Case_filtered_15_540 = Case_filtered_15_540.head()
Test_Control_filtered_15_540 = Control_filtered_15_540.head()
print(Test_Case_filtered_15_540)
print(Test_Control_filtered_15_540)

In [ ]:
#generic function get cohort 
def df_to_cohort(df):
    mrns = list(df.index.values)
    condition = MRNs(mrns)
    return Cohort(condition)

In [ ]:
#get Case Cohort
cohort = df_to_cohort(Case_filtered_15_540)

conditions = (Diagnosis("39%", "ICD-9")|
              Diagnosis("40%", "ICD-9")|
              Diagnosis("42%", "ICD-9")|
              Diagnosis("43%", "ICD-9")|
              Diagnosis("44%", "ICD-9")|
              Diagnosis("45%", "ICD-9")|
             Diagnosis("I0%", "ICD-10")|
             Diagnosis("I2%", "ICD-10")|
             Diagnosis("I3%", "ICD-10")|
             Diagnosis("I4%", "ICD-10")|
              Diagnosis("I5%", "ICD-10")|
              Diagnosis("I6%", "ICD-10")|
              Diagnosis("I7%", "ICD-10")|
              Diagnosis("I8%", "ICD-10")|
              Diagnosis("I9%", "ICD-10"))
Case_filtered_15_540_Cardiovascular = cohort.get(conditions)

In [ ]:
#get Control Cohort
cohort = df_to_cohort(Control_filtered_15_540)

conditions = (Diagnosis("39%", "ICD-9")|
              Diagnosis("40%", "ICD-9")|
              Diagnosis("42%", "ICD-9")|
              Diagnosis("43%", "ICD-9")|
              Diagnosis("44%", "ICD-9")|
              Diagnosis("45%", "ICD-9")|
             Diagnosis("I0%", "ICD-10")|
             Diagnosis("I2%", "ICD-10")|
             Diagnosis("I3%", "ICD-10")|
             Diagnosis("I4%", "ICD-10")|
              Diagnosis("I5%", "ICD-10")|
              Diagnosis("I6%", "ICD-10")|
              Diagnosis("I7%", "ICD-10")|
              Diagnosis("I8%", "ICD-10")|
              Diagnosis("I9%", "ICD-10"))
Control_filtered_15_540_Cardiovascular = cohort.get(conditions)

In [ ]:

Case_filtered_15_540_Cardiovascular

In [ ]:
Control_filtered_15_540_Cardiovascular

In [ ]:
#save diagnosis cohorts
Case_filtered_15_540_Cardiovascular.to_parquet('Case_filtered_15_540_Cardiovascular.parquet')
Control_filtered_15_540_Cardiovascular.to_parquet('Control_filtered_15_540_Cardiovascular.parquet')

In [ ]:
#filter dataframe by one ICD code:
test=Control_filtered_15_540_Cardiovascular[Control_filtered_15_540_Cardiovascular.values == "I81"]
#Wie kann ich nach einem String Teil suchen ? 
print(test)

In [ ]:
#find out whether a patient has a certain disease
def get_mrn_has_certain_diagnosis(mrn, df):
    if (df[df.values == mrn].empty==False): 
        #print("ja hat die Diagnose")
        return 1
    else: 
        #print ("hat sie nicht")
        return 0
    

In [ ]:
#test of func
if (get_mrn_has_certain_diagnosis("414094579",test)):
    print('Juhu')

In [ ]:
Control_MRNs=Control_filtered_15_540.index
Control_MRNs

In [ ]:
y=[]
for mrn in Control_MRNs: 
    x = get_mrn_has_certain_diagnosis(mrn,test)
    a = [mrn, x]
    y.append(a)
    

In [ ]:
y.index(['4140974579', True])

In [ ]:
y[462452]

In [ ]:
col_names = ['MRN','has Diagnose x']
test_df=pd.DataFrame(y,columns=col_names)

In [ ]:
test_df

In [ ]:
def get_has_certain_diagnosis(icd, coding_system, df_mrn, name_df):
    #get cohort
    cohort = df_to_cohort(df_mrn)
    #create condition
    condition= Diagnosis(icd, coding_system)
    #get cohort with condition
    cohort_condition = cohort.get(condition)
    print(cohort_condition)
    #save cohrt
    cohort_condition.to_parquet(name_df)
    # get MRNs of input file 
    cohort_indexes= df_mrn.index
    #get 0 for no diagnosis and 1 for has diagnosis
    cohort_mrn_diagnosis=[]
    for mrn in cohort_indexes: 
        x = get_mrn_has_certain_diagnosis(mrn,cohort_condition)
        a = [mrn, x]
        cohort_mrn_diagnosis.append(a)
    #convert list cohort_mrn_diagnosis to panda
    condition_name= 'has diagnosis '+icd
    col_names = ['MRN',condition_name]
    df_final=pd.DataFrame(cohort_mrn_diagnosis,columns=col_names)
    print(df_final)
    

In [ ]:
get_has_certain_diagnosis("I8%","ICD-10",Test_Case_filtered_15_540,'Test_Case_I87.2')